<a href="https://colab.research.google.com/github/Naps284/Sliding_puzzle_solver_Simple/blob/main/Sliding_puzzle_solver_Simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import heapq
from typing import List, Tuple, Optional
import itertools

class Board:
    def __init__(self, tiles: Tuple[int, ...], dimension: int = 3):
        if len(tiles) != dimension * dimension:
            raise ValueError("Number of tiles must equal dimension^2")
        self.tiles = tiles
        self.dimension = dimension
        self.blank_index = self.tiles.index(0)

    def __eq__(self, other):
        return isinstance(other, Board) and self.tiles == other.tiles and self.dimension == other.dimension

    def __hash__(self):
        return hash((self.tiles, self.dimension))

    def is_goal(self) -> bool:
        # Goal state: 1, 2, 3, ..., (n^2 - 1), 0
        return self.tiles == tuple(list(range(1, self.dimension * self.dimension)) + [0])

    def manhattan(self) -> int:
        # Sum of Manhattan distances of each tile from its goal position.
        distance = 0
        for index, tile in enumerate(self.tiles):
            if tile == 0:
                continue
            target_row = (tile - 1) // self.dimension
            target_col = (tile - 1) % self.dimension
            current_row = index // self.dimension
            current_col = index % self.dimension
            distance += abs(current_row - target_row) + abs(current_col - target_col)
        return distance

    def neighbors(self) -> List['Board']:
        # Generate boards reachable by sliding an adjacent tile into the blank.
        neighbors = []
        row, col = divmod(self.blank_index, self.dimension)
        moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for drow, dcol in moves:
            new_row = row + drow
            new_col = col + dcol
            if 0 <= new_row < self.dimension and 0 <= new_col < self.dimension:
                new_blank_index = new_row * self.dimension + new_col
                new_tiles = list(self.tiles)
                # Swap the blank with the adjacent tile.
                new_tiles[self.blank_index], new_tiles[new_blank_index] = new_tiles[new_blank_index], new_tiles[self.blank_index]
                neighbors.append(Board(tuple(new_tiles), self.dimension))
        return neighbors

    def __str__(self):
        # Pretty-print the board (using a blank space for the 0)
        s = ""
        for i in range(self.dimension):
            row_tiles = self.tiles[i * self.dimension:(i + 1) * self.dimension]
            s += " ".join(str(tile) if tile != 0 else " " for tile in row_tiles) + "\n"
        return s

def is_solvable(tiles: Tuple[int, ...], dimension: int) -> bool:
    # Count inversions (ignoring the blank).
    inv_count = 0
    tile_list = [tile for tile in tiles if tile != 0]
    for i in range(len(tile_list)):
        for j in range(i + 1, len(tile_list)):
            if tile_list[i] > tile_list[j]:
                inv_count += 1

    if dimension % 2 == 1:
        # For odd-dimension puzzles, solvable if inversion count is even.
        return inv_count % 2 == 0
    else:
        # For even-dimension puzzles, count the row of the blank from the bottom.
        blank_index = tiles.index(0)
        blank_row_from_top = blank_index // dimension
        blank_row_from_bottom = dimension - blank_row_from_top
        # Puzzle is solvable if (inversions + blank_row_from_bottom) is odd.
        return (inv_count + blank_row_from_bottom) % 2 == 1

def solve(initial: Board) -> Optional[List[Board]]:
    if not is_solvable(initial.tiles, initial.dimension):
        print("Puzzle is not solvable.")
        return None

    frontier = []
    counter = itertools.count()  # Unique tie-breaker
    initial_priority = initial.manhattan()
    heapq.heappush(frontier, (initial_priority, 0, next(counter), initial, [initial]))
    explored = set()

    while frontier:
        priority, moves, _, board, path = heapq.heappop(frontier)
        if board in explored:
            continue
        explored.add(board)
        if board.is_goal():
            return path
        for neighbor in board.neighbors():
            if neighbor in explored:
                continue
            new_moves = moves + 1
            new_priority = new_moves + neighbor.manhattan()
            new_path = path + [neighbor]
            heapq.heappush(frontier, (new_priority, new_moves, next(counter), neighbor, new_path))

    return None

def main():
    # Get puzzle dimension from the user.
    try:
        dimension = int(input("Enter the dimension of the puzzle (e.g., 3 for 3x3, 4 for 4x4): "))
    except ValueError:
        print("Invalid dimension!")
        return

    expected_tile_count = dimension * dimension
    print(f"Enter the {expected_tile_count} numbers (use 0 for the blank) separated by spaces:")
    tiles_input = input()

    try:
        initial_tiles = tuple(map(int, tiles_input.strip().split()))
    except ValueError:
        print("Invalid input. Please enter numbers only.")
        return

    if len(initial_tiles) != expected_tile_count:
        print(f"Invalid number of tiles. Expected {expected_tile_count} numbers.")
        return

    initial_board = Board(initial_tiles, dimension)
    print("\nInitial Board:")
    print(initial_board)

    if not is_solvable(initial_tiles, dimension):
        print("The puzzle is unsolvable.")
        return

    solution = solve(initial_board)
    if solution:
        move_count = len(solution) - 1
        print(f"Solution found in {move_count} moves:")
        # For each consecutive pair of boards, determine the move (tile number that moved).
        for i in range(move_count):
            current_board = solution[i]
            next_board = solution[i + 1]
            # The blank moves from current_board.blank_index to next_board.blank_index.
            # The tile that moved is the one that was at next_board.blank_index in current_board.
            tile_moved = current_board.tiles[next_board.blank_index]
            print(f"{i + 1}. move {tile_moved}")
    else:
        print("No solution exists.")

if __name__ == "__main__":
    main()


Enter the dimension of the puzzle (e.g., 3 for 3x3, 4 for 4x4): 5
Enter the 25 numbers (use 0 for the blank) separated by spaces:
15 8 9 20 10 6 19 17 18 24 3 12 7 22 13 1 21 14 16 5 23 2 0 4 11

Initial Board:
15 8 9 20 10
6 19 17 18 24
3 12 7 22 13
1 21 14 16 5
23 2   4 11



KeyboardInterrupt: 